In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [61]:
from autorocks.data.loader.all_models_result_aggregator import create_all_models_comparison_dataset
from autorocks.dir_struct import PackageRootDir
from sysgym.envs.rocksdb import schema

param_space = schema.RocksDB10Params()

all_model = create_all_models_comparison_dataset(
    PackageRootDir / "ProcessedDataNew/rocksdb/iops/zippy_workload_15min_turbo/10params/100iter", save_results = False)

default = create_all_models_comparison_dataset(
    PackageRootDir / "ProcessedDataNew/rocksdb/iops/zippy_workload_15min/10_params/10_iter", save_results = False)

In [62]:
import autorocks.viz.viz as viz

model_comparison_data_c = viz.unify_model_name(all_model)


In [63]:
import re
import pandas as pd

perf_df = model_comparison_data_c.sys_observations
perf_df = pd.concat([perf_df, default.sys_observations], ignore_index = True)
perf_df.rename(columns = lambda x: re.sub(r"^db_bench\.\w*\.", "", x), inplace = True)
color_palette = viz.create_color_palette_df(
    perf_df, ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
)
perf_df[['iteration', 'step', 'iops', 'model']]

In [64]:


main_target = "iops"


iops_box = viz.perf_boxplot(
    perf_df = perf_df,
    optimization_type = viz.OptimizationType.MAXIMIZE,
    ylabel = "IOPS",
    comparison_col = main_target,
    model_palette_map = color_palette,
    horizontal_line = "Default",
    fig_size = (9, 5),
    add_roi = True
)

iops_box

In [65]:
iops_box.savefig("/Users/salabed/workspace/latex_writings/thesis/phd_dissertation/Chapters/BoGraphEval/figures/rocksdb_turbo_iops.pdf", dpi= 300)
iops_box.savefig("/Users/salabed/workspace/latex_writings/thesis/phd_dissertation/Chapters/BoGraphEval/figures/rocksdb_turbo_iops.svg", dpi= 300)



In [66]:
model_performance = model_comparison_data_c.model_performance
model_performance.rename(columns = lambda x: re.sub(r"^db_bench\.\w*\.", "", x), inplace = True)
color_palette = viz.create_color_palette_df(
    model_performance, ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
)
model_performance

In [67]:
color_palette

In [68]:
viz.model_perf_plot(model_perf_df=model_performance,
comparison_col='inference_time',
ylabel="Auto-tuner wall-time in seconds")



In [69]:
model_performance['relative_time'] = model_performance['inference_time'] / perf_df['exe_time']
viz.model_perf_plot(model_perf_df=model_performance,
comparison_col='relative_time',
ylabel="Wall-time relative to RocksDB in seconds")


In [70]:
viz.model_perf_plot(model_perf_df=perf_df,
comparison_col='exe_time',
ylabel="RocksDB execution time")



In [71]:
iops_box = viz.roi_boxplot(
    df = perf_df,
    optimization_type = viz.OptimizationType.MAXIMIZE,
    ylabel = "IOPS",
    model_baseline = "Default",
    comparison_col = main_target,
    model_palette_map = color_palette,
    fig_size = (4, 3)
)
iops_box

In [ ]:

convergence_df = perf_df.copy()
convergence_df["rolling"] = (
    convergence_df[[main_target, "model", "iteration"]]
    .groupby(["model", "iteration"])
    [main_target]
    .agg(f"cum{str(viz.OptimizationType.MAXIMIZE)}")
)


In [79]:

fig = viz.convergence_lineplot(
    df=convergence_df,
    optimization_type=viz.OptimizationType.MAXIMIZE,
    ylabel=main_target,
    column_name="rolling",
    model_palette_map=color_palette,
    fig_size=(9, 5),
    horizontal_line="Default",
    xlim = [10, 101]
)
fig

In [81]:
fig.savefig("/Users/salabed/workspace/latex_writings/thesis/phd_dissertation/Chapters/BoGraphEval/figures/rocksdb_turbo_iops_conv.pdf", dpi= 300)
fig.savefig("/Users/salabed/workspace/latex_writings/thesis/phd_dissertation/Chapters/BoGraphEval/figures/rocksdb_turbo_iops_conv.svg", dpi= 300)



In [74]:
convergence_df[main_target]